In [8]:
# General purpose libraries
import logging
import os

# Document loading and splitting
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Tokenizer for the text processing
from transformers import AutoTokenizer

# Embeddings and vector storage
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

import faiss 

In [9]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [10]:
EMBEDDING_MODEL_NAME = "thenlper/gte-small"

In [11]:
# Path to the document 
file_path = "../data/raw/TA-9-2024-0138_EN.pdf" 

# Function to split the document into chunks
def split_document_into_chunks(file_path: str, chunk_size: int, tokenizer_name: str = EMBEDDING_MODEL_NAME):
    """
    Load a document and split it into smaller chunks for processing.

    Args:
        file_path (str): Path to the document file.
        chunk_size (int): The maximum size of each chunk (number of tokens).
        tokenizer_name (str): The name of the tokenizer to use for splitting the document.

    Returns:
        List of split document chunks.
    """
    # Check if the document file exists
    if not os.path.isfile(file_path):
        logging.error(f"The file '{file_path}' does not exist.")
        return None

    # Load the document using PyPDFLoader
    loader = PyPDFLoader(file_path)
    pages = loader.load()
    logging.info(f"The document has been loaded successfully. Total number of pages: {len(pages)}.")

    # Initialize a text splitter
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size * 0.1),  # 10% overlap between chunks
        add_start_index=True, 
        strip_whitespace=True
    )

    chunks = text_splitter.split_documents(pages)
    logging.info(f"The document has been split into {len(chunks)} chunks.")

    return chunks

# Split the document into chunks
chunk_size = 128  
chunks = split_document_into_chunks(file_path, chunk_size)

2024-10-22 14:47:29,983 - INFO - The document has been loaded successfully. Total number of pages: 459.
2024-10-22 14:47:31,798 - INFO - The document has been split into 1391 chunks.


In [12]:
# Function to generate embeddings for the document chunks
def generate_embeddings(chunks):
    """
    Generate embeddings for the given document chunks and store them using FAISS (uses the nearest neighbor search algorithm).
    
    Args:
        chunks (list): List of document chunks to generate embeddings for.
        
    Returns:
        FAISS index containing the document embeddings.
    """
    embedding_model = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL_NAME,
        multi_process=True,
        model_kwargs={"device": "cpu"},  # Use CPU for embeddings
        encode_kwargs={"normalize_embeddings": True}
    )
    logging.info(f"Embedding model '{EMBEDDING_MODEL_NAME}' loaded successfully.")

    KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
        chunks, embedding_model, distance_strategy=DistanceStrategy.COSINE
    )
    logging.info("Embeddings generated successfully.")

    return KNOWLEDGE_VECTOR_DATABASE

# Generate embeddings for the document chunks
if chunks is not None:
    KNOWLEDGE_VECTOR_DATABASE = generate_embeddings(chunks)
else:
    logging.error("Chunks not generated. Please check the document splitting process.")

2024-10-22 14:47:31,807 - INFO - Load pretrained SentenceTransformer: thenlper/gte-small
2024-10-22 14:47:33,660 - INFO - Embedding model 'thenlper/gte-small' loaded successfully.
2024-10-22 14:47:33,662 - INFO - CUDA/NPU is not available. Starting 4 CPU workers
2024-10-22 14:47:33,662 - INFO - Start multi-process pool on devices: cpu, cpu, cpu, cpu
2024-10-22 14:48:23,253 - INFO - Embeddings generated successfully.


In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

READER_MODEL_NAME = "Qwen/Qwen2-0.5B"
tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(READER_MODEL_NAME, device_map="auto", torch_dtype="auto")

# Initialize the pipeline for text generation using the new model
READER_LLM = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,  
    do_sample=True,
    temperature=0.7,
    repetition_penalty=1.2,
    return_full_text=False,
)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

C:\Users\eduar\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\eduar\.cache\huggingface\hub\models--Qwen--Qwen2-0.5B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Error while downloading from https://huggingface.co/Qwen/Qwen2-0.5B/resolve/main/tokenizer.json: HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out.
Trying to resume download...
2024-10-22 14:48:36,730 - WARNING - Error while downloading from https://huggingface.co/Qwen/Qwen2-0.5B/resolve/main/tokenizer.json: HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out.
Trying to resume download...


tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [14]:
user_query = "What is the purpose of this Regulation?"
print(f"\nStarting retrieval for {user_query=}...")
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=5)
print(
    "\n==================================Top document=================================="
)
print(retrieved_docs[0].page_content)
print("==================================Metadata==================================")
print(retrieved_docs[0].metadata)

2024-10-22 14:51:26,780 - INFO - CUDA/NPU is not available. Starting 4 CPU workers
2024-10-22 14:51:26,780 - INFO - Start multi-process pool on devices: cpu, cpu, cpu, cpu



Starting retrieval for user_query='What is the purpose of this Regulation?'...

==================================Top document==================================
as required by this Regulation.
==================================Metadata==================================
{'source': '../data/raw/TA-9-2024-0138_EN.pdf', 'page': 74, 'start_index': 1570}


In [39]:
retrieved_docs_text = [
    doc.page_content for doc in retrieved_docs
]  # We only need the text of the documents
context = "\nExtracted documents:\n"
context += "".join(
    [f"Document {str(i)}:::\n" + doc for i, doc in enumerate(retrieved_docs_text)]
)

final_prompt = f"""
Context:
{context}

Question: {user_query}
"""

# Redact an answer
generated_text = READER_LLM(final_prompt, truncation=True, max_new_tokens=1000)

# print("\nGenerated Text:\n", generated_text)

# Assuming this is the relevant portion of the output
output = generated_text[0]['generated_text']  # Extract the generated text from the dictionary

# Step 1: Split by the '<|im_start|>assistant' to find where the assistant's answer begins
answer_start = output.split('<|im_start|>assistant')

# Step 2: Extract the first occurrence of the assistant's response
if len(answer_start) > 1:
    first_answer = answer_start[1].split('<|im_end|>')[0].strip()  # Take the first answer and strip extra spaces

print(first_answer)

Both `max_new_tokens` (=1000) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


According to the original text of Document 5, the purpose of this Regulation is to improve the functioning of the internal market and promote the uptake of human-centric and trustworthy AI (AI). Specifically, it aims to ensure that these technological developments benefit society, protect individuals' privacy and security, respect human dignity, and uphold democratic values such as democracy, freedom of expression, and the right to information and access. Additionally, the regulation promotes transparency and accountability through clear and accessible guidelines and regulations aimed at preventing and addressing risks associated with the adoption and usage of AI technology.
